# Challenge

The challenge is to find a combination of items from Zalando's catalog, given the price of each item and a list of packages including a couple of items each time (but one item only once) such that the capacity of 40 liters is not exceeded. Goal has no limit on price but rather to maximize the total price with the optimal items selected.

The measured volume of packages isn't accurate, but rather has a mean of 0 and a variance of 2 thus following a normal distribution. 

# Solution

The proposed solution is a two step process as follows

## Get volume estimates of items

Since it is known that the package measurements all have a mean of 0 and a variance of 2, it forms an advantage to use least squares to estimate the individual item volumes.
- Mean of 0 -> errors are unbiased. if the mean wasn't 0, our estimates would've been biased
- Variance of 2 -> a constant variance across all measurements means that the precision is consistent and estimated would be unbiased
- Normal Distribution -> The least square estimates are also the maximum likelihood estimates

## Solve a 0/1 knapsack problem for the given capacity

A 0/1 knapsack problem includes selecting items (only once) in a combination that maximizes the value while still laying within the sack limit. This is a typical question asked in programming interviews and is expected a Dynamic Programming solution. But just for a twist, this problem has been solved in 2 ways
- __Integer Programming__ -> the problem is formulated as an integer programming problem
- __Sparse Dynamic Programming__ -> Instead of having a discrete linear space of the volume/ value, this method only considers sparsely achievable volumes! meaning, for a capacity of 40L, one doesn't have to create a table from 0->40 with a space of 1

In [1]:
# We shall import necessary libraries for the challenge

import numpy as np
import pandas as pd
import warnings as warn
from sklearn.linear_model import LinearRegression, SGDRegressor
from scipy.optimize import milp, Bounds, LinearConstraint

In [2]:
# Loading the items and packages data as pandas dataframes.
# In the entire code, a pandas dataframe is written with a suffix _df
items_df = pd.read_json('items.json')
packages_df = pd.read_json('packages.json')

In [3]:
# Info about items dataframe

print('Items Dataframe Info:\n')
print(items_df.info())
print('Items Dataframe head:\n')
print(items_df.head())

Items Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    60 non-null     object
 1   price   60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None
Items Dataframe head:

  name  price
0   A1     98
1   A2    108
2   A3     82
3   A4     34
4   A5     52


In [4]:
unique_items = np.sort(items_df['name'].unique().flatten())
print('Total number of items: {}'.format(unique_items.size))
print('Item names: \n{}'.format(unique_items))

Total number of items: 60
Item names: 
['A1' 'A10' 'A11' 'A12' 'A13' 'A14' 'A15' 'A16' 'A17' 'A18' 'A19' 'A2'
 'A20' 'A21' 'A22' 'A23' 'A24' 'A25' 'A26' 'A27' 'A28' 'A29' 'A3' 'A30'
 'A31' 'A32' 'A33' 'A34' 'A35' 'A36' 'A37' 'A38' 'A39' 'A4' 'A40' 'A41'
 'A42' 'A43' 'A44' 'A45' 'A46' 'A47' 'A48' 'A49' 'A5' 'A50' 'A51' 'A52'
 'A53' 'A54' 'A55' 'A56' 'A57' 'A58' 'A59' 'A6' 'A60' 'A7' 'A8' 'A9']


In [5]:
# Info about packages dataframe

print('Packages Dataframe Info:\n')
print(packages_df.info())
print('Packages Dataframe head:\n')
print(packages_df.head())

Packages Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_volume  1000 non-null   float64
 1   items         1000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 15.8+ KB
None
Packages Dataframe head:

   total_volume                                    items
0         36.04                      [A28, A3, A33, A59]
1        123.78  [A51, A23, A57, A5, A33, A28, A12, A47]
2         58.54                          [A10, A19, A22]
3         79.33           [A38, A28, A57, A27, A46, A45]
4         54.36                          [A40, A24, A57]


In [6]:
unique_items_included_in_packages = np.sort(packages_df['items'].explode().unique().flatten())
print('Total number of unique items included in packages: {}'.format(unique_items_included_in_packages.size))
print('Item names included in packages: \n{}'.format(unique_items_included_in_packages))

Total number of unique items included in packages: 60
Item names included in packages: 
['A1' 'A10' 'A11' 'A12' 'A13' 'A14' 'A15' 'A16' 'A17' 'A18' 'A19' 'A2'
 'A20' 'A21' 'A22' 'A23' 'A24' 'A25' 'A26' 'A27' 'A28' 'A29' 'A3' 'A30'
 'A31' 'A32' 'A33' 'A34' 'A35' 'A36' 'A37' 'A38' 'A39' 'A4' 'A40' 'A41'
 'A42' 'A43' 'A44' 'A45' 'A46' 'A47' 'A48' 'A49' 'A5' 'A50' 'A51' 'A52'
 'A53' 'A54' 'A55' 'A56' 'A57' 'A58' 'A59' 'A6' 'A60' 'A7' 'A8' 'A9']


With the above information, we can set up a system of linear equations $Ax=b$

where $A$ is a matrix of shape $N_{packages} \times N_{items}$, $b$ is a matrix of shape $N_{packages} \times 1$ resembling package volumes and let $x$ be the variables i.e. item volumes to be solved for. $x$ is a matrix of shape $N_{items} \times 1$

In [7]:
items_not_included_in_packages = list(set(unique_items)-set(unique_items_included_in_packages))
num_items_not_included_in_packages = len(items_not_included_in_packages)

print('Items not included in packages: \n{}'.format(items_not_included_in_packages))
print('Number of items not included in packages: \n{}'.format(num_items_not_included_in_packages))

Items not included in packages: 
[]
Number of items not included in packages: 
0


The number of equations (1000) are more than the number of variables (60), which is an overdetermined system.

Since it is clear that all the variables i.e. item prices are included into the packages as shown above, it should be possible to approximately estimate price of every item. In case any item is omitted from the packages dataset, we need to take further steps to deal with the edge case, which I can freely discuss in person in an interview

In [8]:
# Generating A and b matrices

item_index_map = {item_name: i_ for i_, item_name in enumerate(unique_items)}

A = np.zeros([len(packages_df),unique_items.size])
b = packages_df['total_volume'].values

for package_i_, package_items in enumerate(packages_df['items']):
    for item_name in package_items:
        if item_name in item_index_map:
            A[package_i_, item_index_map[item_name]] = 1

### Least Squares/ Linear Regression
A way to solve the system of overdetermined linear equations is least squares which minimizes the Euclidean norm of the residual vector $||Ax-b||$ as where $x=(A^{T}A)^{-1}A^{T}b$

In [9]:
with warn.catch_warnings():
    warn.simplefilter("ignore", RuntimeWarning)
    estimated_volumes_ls = (np.linalg.inv(A.T@A)@A.T)@b
    print(estimated_volumes_ls)

[22.02111852 14.54056646 22.77823394 14.96404077 21.54299888 29.66981803
 11.03659771 15.9578205   6.86438901  8.35269535 15.18408573 14.45781057
 22.10481856 26.43536529 25.80303081  7.27123907 25.77582664  9.48204202
 20.5833954   7.51540813  6.8321253  20.85050277  8.45491676 19.32663857
  6.87568259  0.92182479  5.94642204 13.24674395  4.97568334 25.11605272
 12.50011259  9.89830518  1.55541432 23.95055876 10.53217932 23.03327846
 11.56637414  5.00575174  4.8114853  22.43858888 17.58217752 26.03734021
  7.21260259 11.17668406 16.60870844 11.38489874 26.99512695 23.01064414
  5.62320838 11.65542281 16.50891537 10.44151953 18.70332939 25.49260732
 16.19378997  0.37567657 12.35544498  8.83333203  3.54618027  0.95934005]


The same result can be obtained by using a Linear Regression implementation of scikit-learn

In [10]:
with warn.catch_warnings():
    warn.simplefilter("ignore", RuntimeWarning)
    linear_model = LinearRegression(positive=True, fit_intercept=False)
    linear_model.fit(A,b)

In [11]:
estimated_volumes_lr = linear_model.coef_
print(estimated_volumes_lr)
item_volume_df = pd.DataFrame({'name': unique_items, 'volume': estimated_volumes_lr})
item_price_volume_df = pd.merge(items_df, item_volume_df, on='name')

[22.02111852 14.54056646 22.77823394 14.96404077 21.54299888 29.66981803
 11.03659771 15.9578205   6.86438901  8.35269535 15.18408573 14.45781057
 22.10481856 26.43536529 25.80303081  7.27123907 25.77582664  9.48204202
 20.5833954   7.51540813  6.8321253  20.85050277  8.45491676 19.32663857
  6.87568259  0.92182479  5.94642204 13.24674395  4.97568334 25.11605272
 12.50011259  9.89830518  1.55541432 23.95055876 10.53217932 23.03327846
 11.56637414  5.00575174  4.8114853  22.43858888 17.58217752 26.03734021
  7.21260259 11.17668406 16.60870844 11.38489874 26.99512695 23.01064414
  5.62320838 11.65542281 16.50891537 10.44151953 18.70332939 25.49260732
 16.19378997  0.37567657 12.35544498  8.83333203  3.54618027  0.95934005]


We could also use iterative methods to give approximate results, but in this case, the final results depend upon a lot of factors such as:
- initial assumtion
- learning rate
- optimizer used
- number of iterations
- tolerance

But in practical cases, when the matrix $A$ grows in size, due to large number of packages and an increase in items in the catalog, it can also be advised to use iterative approximate methods. 

But for the time being, we are satisfied with the least-squared implementation above


Now we have estimates of the item volumes, we can attach it to the existing dataframe and proceed further to solve the problem of maximuzing price while staying within the volume limit

In [12]:
print('Item Price Volume Dataframe Info:\n')
print(item_price_volume_df.info())
print('Item Price Volume Dataframe head:\n')
print(item_price_volume_df.head())

Item Price Volume Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    60 non-null     object 
 1   price   60 non-null     int64  
 2   volume  60 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ KB
None
Item Price Volume Dataframe head:

  name  price     volume
0   A1     98  22.021119
1   A2    108  14.457811
2   A3     82   8.454917
3   A4     34  23.950559
4   A5     52  16.608708


The problem to solve is a classic 0/1 knapsack problem. Where given the capacity of a sack, one has to fill it with items which result in maximum value (here price) while staying within the limits of the sack capacity. There are several ways to solve it, I will provide two:

### Integer Programming

The problem can be formulated as an Integer Programming problem as follows:

\begin{align*}
\text{Minimize: } & -c^T x \\
\text{Subject to: } & A x \le b \\ 
& x_i \in \{0, 1\} \quad \forall i \in I
\end{align*}


The above formulation, when solved for $x$ will give a linear combination of items, with maximum price and volume lying within the sack capacity. The integer constraints and bounds make sure that either one item is selected once or zero number of times

In [13]:
def milp_solve(item_price_volume_df, capacity):
    """
    Solves the 0/1 Knapsack problem using a integer programming approach.

    Args:
        item_price_volume_df (pd.DataFrame): DataFrame with 'name', 'price', 'volume' columns.
        capacity (float): The maximum volume capacity of the knapsack.

    Returns:
        dict: {
        'error' (boolean): Flag informing error within the MILP solver,
        'result' (pd.DataFrame): Dataframe with 'name', 'price', 'volume' columns.
        }
    """
    
    num_items = len(item_price_volume_df)
    
    # objective function
    c = -1*item_price_volume_df['price'].values

    # constraints
    # Ax <= b
    # sum of item volumes should be less than or equal to the sack capacity
    A = item_price_volume_df['volume'].values.reshape(1, num_items)
    constraints = LinearConstraint(A, [0], [capacity])

    # the number of times each item can be chosen lies between 0 and 1
    bounds = Bounds(0, 1)

    # integer constraints
    # if an item is chosen, it is chosen ONCE as an entity, not in fractions!
    integrality = np.ones(num_items, dtype=int)

    result = milp(c, constraints=constraints, bounds=bounds, integrality=integrality)

    if result.success:
        selected_items_indices_ = np.where(result.x.round()==1)[0]
        selected_items = item_price_volume_df.iloc[selected_items_indices_]
        return {
            'error': False,
            'result': selected_items
        }
    else:
        return {'error': True}

In [14]:
milp_res = milp_solve(item_price_volume_df, 40.0)
if not milp_res['error']:
    selected_items_df = milp_res['result']
    print('Selected items: {}'.format(selected_items_df['name'].values))
    print('Total price of selected items: {}'.format(selected_items_df['price'].values.sum()))
    print('Total volume of selected items: {} L'.format(selected_items_df['volume'].values.sum()))

Selected items: ['A6' 'A8' 'A9' 'A23' 'A32' 'A35' 'A38' 'A44' 'A48']
Total price of selected items: 757
Total volume of selected items: 39.97233714874501 L


As you see, the above items are selected with the maximum price of 757 and 39.97 Liters, which satisfies our limit of 40 litres

### Dynamic Programming (Sparse, worthy a discussion)

It is quite well known that a knapsack problem with dynamic programming has been one of the typical questions asked in a programming interview. Here I will implement it with a twist!

In a typical DP problem, the user creates a discrete linspace of volume to create huge matrix! When the volume is a floating point, the user creates a linspace of value, in order to maximize value. But the problem here is always to create an incremental linspace of either volume or value. Why?

I am presenting here an approach which only considers achievable volume and not a discrete linspace. Whether it's efficient or not can be discussed in person, but I thought it would be impressive to do it.

In [15]:
def solve_knapsack_sparse_dp(item_price_volume_df, capacity, significant_digits=2):
    """
    Solves the 0/1 Knapsack problem using a sparse (dictionary-based) dDP approach.

    Args:
        item_price_volume_df (pd.DataFrame): DataFrame with 'name', 'price', 'volume' columns.
        capacity (float): The maximum volume capacity of the knapsack.
        significant_digits (int): Significant digits to be considered in volume calculation. E.g. with 3 significant_digits, vol of 2.4586 is rounded off to 2.459

    Returns:
        tuple: (max_price, selected_items_names, actual_volume_used)
    """

    scale_factor = 10**significant_digits
    scaled_capacity = int(np.round(capacity, significant_digits) * scale_factor)

    # dp: A dict with keys as achievable scaled volumes, and values are the corresp. max prices
    # We also store which items contribute to that volume for reconstruction
    dp = {0: {'price': 0, 'items': []}} 

    for index, row in item_price_volume_df.iterrows():
        # for each item
        
        item_name = row['name']
        item_price = row['price']
        item_volume = row['volume']
        
        # ensuring at least 1 as scaled volume
        scaled_item_volume = max(1, int(item_volume * scale_factor)) 
        
        for current_scaled_vol, data in list(dp.items()):
            # loop over existing calculated volumes and respective items' combination
            
            current_price = data['price']
            current_items = data['items']

            new_scaled_vol = current_scaled_vol + scaled_item_volume
            new_price = current_price + item_price

            if new_scaled_vol <= scaled_capacity:
                # if new volume is still less than the capacity of the sack
                if new_scaled_vol not in dp or new_price > dp[new_scaled_vol]['price']:
                    # if the new volume isnt recorded yet or the new volume's price is greater than prev max price
                    # record it in dp dict
                    dp[new_scaled_vol] = {'price': new_price, 'items': current_items + [item_name]}

    # calculate the max price, correponding volume and item combination
    max_price = 0
    best_scaled_volume = 0
    selected_items = []
    
    for scaled_vol, data in dp.items():
        if data['price'] > max_price:
            max_price = data['price']
            best_scaled_volume = scaled_vol
            selected_items = data['items']
        
    actual_volume_used = item_price_volume_df[item_price_volume_df['name'].isin(selected_items)]['volume'].sum()

    return max_price, selected_items, float(actual_volume_used)

In [16]:
print(solve_knapsack_sparse_dp(item_price_volume_df, 40))

(757, ['A6', 'A8', 'A9', 'A23', 'A32', 'A35', 'A38', 'A44', 'A48'], 39.97233714874501)


The sparse dynamic programming solution of the above problem is also same as the one obtained by linear integer programming.

## Solution

Thus, Ahmad can buy (A6, A8, A9, A23, A32, A35, A38, A44, A48) for Sergey's birthday. The estimated volume of items is 39.972 Litres and total price is 757.